In [2]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver

# 팔로워 수 크롤링

In [35]:
driver = webdriver.Chrome('chromedriver')
driver.get("https://www.instagram.com/accounts/login/")
driver.implicitly_wait(10)

# 로그인
username = 'gyeonggi_korea'
password = 'zhsxpscm123!'

element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
element_id.send_keys(username)
element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
element_password.send_keys(password)
driver.implicitly_wait(3)

##로그인버튼 클릭
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

<ipython-input-35-985c6fd20ebc>:9: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
<ipython-input-35-985c6fd20ebc>:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
<ipython-input-35-985c6fd20ebc>:16: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()


In [36]:
url = 'https://www.instagram.com/gyeonggi_korea/'
# webdriver 로드
driver.get(url)

# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
soup.title

<title>gyeonggi_korea(경기도)(@gyeonggi_korea) • Instagram 사진 및 동영상</title>

In [37]:
post_link = []
div = soup.find_all('div', class_="Nnq7C weEfm")

# 게시글 줄 순서(1번째부터 몇 번째 줄까지 가져올 것인지)
number_of_line = 2
# 게시글 링크 3개씩 가져오기
for row in range(number_of_line):
    a_tags = div[row].find_all('a')
    for i in range(3):
        link = a_tags[i].attrs['href']
        post_link.append(link)
# 수집한 링크 개수 확인
print("수집한 링크의 개수 : ", len(post_link))
print("수집한 링크 주소")
print(post_link)

수집한 링크의 개수 :  6
수집한 링크 주소
['/p/CamCGL3LSHX/', '/p/Cag862TrWuX/', '/p/CaWWan2BoZc/', '/p/CaRgX_svAjY/', '/p/CaG5pIbv7MV/', '/p/CZ9rGIHP423/']


In [38]:
link_list = []
num_of_post = len(post_link)
for i in (range(num_of_post)):
    link_list.append("https://www.instagram.com" + post_link[i])
link_list

['https://www.instagram.com/p/CamCGL3LSHX/',
 'https://www.instagram.com/p/Cag862TrWuX/',
 'https://www.instagram.com/p/CaWWan2BoZc/',
 'https://www.instagram.com/p/CaRgX_svAjY/',
 'https://www.instagram.com/p/CaG5pIbv7MV/',
 'https://www.instagram.com/p/CZ9rGIHP423/']

In [39]:
for i in range(1):
    # 게시물 타이틀 수집
    url = link_list[i]
    driver.get(url)
    soup = bs(driver.page_source, 'lxml')
    print(soup.title.text)
    driver.implicitly_wait(3)

Instagram의 gyeonggi_korea(경기도)님: “A complete guide to unique cafés in Yongin City of Gyeonggi Province

📢Real Garden K
Feel like having a ‘forest and phytoncide’ day?…”


In [52]:
# 게시물 좋아요 수 수집
a_page = bs(driver.page_source, 'lxml')
like_data = a_page.find_all('div', class_='_7UhW9 xLCgt qyrsm KV-D4 fDxYl T0kll')
number_of_like = like_data[1].find('span')
number_of_like = int(number_of_like.text)
number_of_like

25

In [40]:
type(like_data)

bs4.element.Tag

In [44]:
num_of_like = int(like_data.text[4:-1])
num_of_like

24

In [46]:
for i in range(1,2):
    url = link_list[i]
    driver.get(url)
    soup = bs(driver.page_source, 'lxml')
    print(soup.title.text)
    time.sleep(0.5)

Instagram의 gyeonggi_korea(경기도)님: “Mystical constellations and planets🌎,
Don’t you want to see them all?
Let’s visit the Anseong Astronomical Science Museum! 🌃

Observe the…”


In [47]:
a_page = bs(driver.page_source, 'lxml')
like_data = a_page.find('div', class_='_7UhW9 xLCgt qyrsm KV-D4 fDxYl T0kll')
like_data.text

'좋아요 24개'

In [48]:
for i in range(3):
    url = link_list[i]
    driver.get(url)
    soup = bs(driver.page_source, 'lxml')
    print(soup.title.text)
    time.sleep(0.5)

    a_page = bs(driver.page_source, 'lxml')
    like_data = a_page.find('div', class_='_7UhW9 xLCgt qyrsm KV-D4 fDxYl T0kll')
    num_of_like = int(like_data.text[4:-1])
    print(num_of_like)

Instagram의 gyeonggi_korea(경기도)님: “A complete guide to unique cafés in Yongin City of Gyeonggi Province

📢Real Garden K
Feel like having a ‘forest and phytoncide’ day?…”
24
Instagram의 gyeonggi_korea(경기도)님: “Mystical constellations and planets🌎,
Don’t you want to see them all?
Let’s visit the Anseong Astronomical Science Museum! 🌃

Observe the…”
24
Instagram의 gyeonggi_korea(경기도)님: “Shall we take a day trip
to nature-friendly Uiwang City of Gyeonggi Province? 🚓

📢Cheonggyesa🙏

A Buddhist temple on picturesque…”
26


# insta 좋아요 수 크롤링

In [90]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver

In [92]:
driver = webdriver.Chrome('chromedriver')
driver.get("https://www.instagram.com/accounts/login/")
driver.implicitly_wait(10)

username = 'gyeonggi_korea'
password = 'zhsxpscm123!'

element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
element_id.send_keys(username)
time.sleep(1)
element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
element_password.send_keys(password)

time.sleep(1.5)

##로그인버튼 클릭
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

<ipython-input-92-b311796cbd22>:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
<ipython-input-92-b311796cbd22>:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
<ipython-input-92-b311796cbd22>:16: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()


In [103]:
url = 'https://www.instagram.com/gyeonggi_korea/'
# webdriver 로드
driver.get(url)

# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
soup.title

<title>gyeonggi_korea(경기도)(@gyeonggi_korea) • Instagram 사진 및 동영상</title>

In [104]:
post_link = []
div = soup.find_all('div', class_="Nnq7C weEfm")

# 게시글 줄 순서(1번째부터 몇 번째 줄까지 가져올 것인지)
number_of_line = 2
# 게시글 링크 3개씩 가져오기
for row in range(number_of_line):
    a_tags = div[row].find_all('a')
    for i in range(3):
        link = a_tags[i].attrs['href']
        post_link.append(link)
# 수집한 링크 개수 확인
print("수집한 링크의 개수 : ", len(post_link))
print("수집한 링크 주소")
print(post_link)

In [105]:
title_list = []
like_list = []
for i in range(3):
    url = link_list[i]
    driver.get(url)
    soup = bs(driver.page_source, 'lxml')
    temp_title = soup.title.text[34:]
    title_list.append(temp_title)
    print(temp_title)
    time.sleep(0.5)

    a_page = bs(driver.page_source, 'lxml')
    like_data = a_page.find('div', class_='_7UhW9 xLCgt qyrsm KV-D4 fDxYl T0kll')
    num_of_like = int(like_data.text[4:-1])

    like_list.append(num_of_like)
    print(num_of_like)

A complete guide to unique cafés in Yongin City of Gyeonggi Province

📢Real Garden K
Feel like having a ‘forest and phytoncide’ day?…”
25
Mystical constellations and planets🌎,
Don’t you want to see them all?
Let’s visit the Anseong Astronomical Science Museum! 🌃

Observe the…”
24
Shall we take a day trip
to nature-friendly Uiwang City of Gyeonggi Province? 🚓

📢Cheonggyesa🙏

A Buddhist temple on picturesque…”
26


In [106]:
import pandas as pd
from datetime import datetime

In [107]:
insta_df = {'좋아요_수':num_of_like, '제목':title_list}
data = pd.DataFrame(insta_df)
data

,좋아요_수,제목
0,26,A complete guide to unique cafés in Yongin Cit...
1,26,"Mystical constellations and planets🌎,\nDon’t y..."
2,26,Shall we take a day trip\nto nature-friendly U...


In [108]:
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'insta_' + file_name + '.csv'
path = './crawling_data/'
data.to_csv(path+file_name, index=False, encoding='utf-8')

In [109]:
url = 'https://business.facebook.com/creatorstudio/instagram_content_posts'
# webdriver 로드
driver.get(url)

In [ ]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')

# 인스타그램 게시물 좋아요 수 크롤링

In [79]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
import pandas as pd
import datetime

In [88]:
def crawling(number_of_line):
    # 사이트에 로그인하기
    driver = webdriver.Chrome('chromedriver')
    driver.get("https://www.instagram.com/accounts/login/")
    driver.implicitly_wait(10)
    # 아이디, 비번
    username = 'gyeonggi_korea'
    password = 'zhsxpscm123!'
    # 아이디, 비번 입력
    element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
    element_id.send_keys(username)
    time.sleep(1)
    element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
    element_password.send_keys(password)
    time.sleep(1.5)

    ##로그인버튼 클릭
    driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()
    time.sleep(1.5)

    # 사이트 접속
    url = 'https://www.instagram.com/gyeonggi_korea/'
    driver.get(url)
    time.sleep(3)
    # 페이지 크롤링
    soup = bs(driver.page_source, 'lxml')

    post_link = []
    div = soup.find_all('div', class_="Nnq7C weEfm")

    # 게시글 줄 순서(1번째부터 몇 번째 줄까지 가져올 것인지)
    number = number_of_line
    # 게시글 링크 3개씩 가져오기
    for row in range(number):
        a_tags = div[row].find_all('a')
        for i in range(3):
            link = a_tags[i].attrs['href']
            post_link.append(link)
    # 수집한 링크 개수 확인
    print("수집한 링크의 개수 : ", len(post_link))
    print("수집한 링크 주소")
    print(post_link)

    # 게시글 타이틀과 좋아요 수 수집
    title_list = []
    like_list = []
    for i in range(len(link_list)):
        # 게시글 타이틀 수집
        url = link_list[i]
        driver.get(url)
        soup = bs(driver.page_source, 'lxml')
        temp_title = soup.title.text[34:]
        title_list.append(temp_title)
        print(temp_title)
        driver.implicitly_wait(1)

        # 게시글 좋아요 수 수집
        like_data = soup.find_all('div', class_='_7UhW9 xLCgt qyrsm KV-D4 fDxYl T0kll')
        number_of_like = like_data[1].find('span')
        number_of_like = int(number_of_like.text)
        like_list.append(number_of_like)
        print("좋아요 수 : ", number_of_like, '개')

    insta_df = {'좋아요_수':like_list, '제목':title_list}
    data = pd.DataFrame(insta_df)

    # 파일로 출력
    file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
    file_name = 'instagram_' + file_name + '.csv'
    path = './crawling_data/instagram/'
    data.to_csv(path+file_name, index=False, encoding='utf-8')

    return data

In [89]:
# 수집할 게시글 줄 수
df = crawling(3)
df

<ipython-input-88-26a342313a75>:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
<ipython-input-88-26a342313a75>:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
<ipython-input-88-26a342313a75>:17: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()


IndexError: list index out of range